# Imports

In [1]:
import xarray as xr
import datatree
import matplotlib.pyplot as plt
from dask.distributed import Client, LocalCluster

In [2]:
# cluster = LocalCluster()  
# client = Client(cluster)
# cluster

# 1. Radar data
Radar data is stored in `xarray datatree` structure using ARCO format. We can access all `sweeps` and all `variables` (dual polarization) by using `datatree.open_datatree` method. We converted three days of continuous radar data from Barrancabermeja radar (04-06-2023 to 04-08-2023)

In [3]:
path = '/data/keeling/a/alfonso8/gpm/zarr/BAR.zarr'

In [4]:
!ls '/data/keeling/a/alfonso8/gpm/zarr/BAR.zarr/'

sweep_0  sweep_2  sweep_4  sweep_6  sweep_8
sweep_1  sweep_3  sweep_5  sweep_7  sweep_9


In [5]:
%%time
dt = datatree.open_datatree(path, engine='zarr', consolidated=True)

/data/keeling/a/alfonso8/miniconda3/envs/xradar/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: user 1.88 s, sys: 1.28 s, total: 3.16 s
Wall time: 8.81 s


In [ ]:
dt

In [6]:
dt['sweep_0'].ds.times.min().values

numpy.datetime64('2023-04-06T00:00:22.709000000')

In [7]:
dt['sweep_0'].ds.times.max().values

numpy.datetime64('2023-04-08T23:50:17.590000000')

In [ ]:
print(f'size: {dt.nbytes / (1024 ** 3)} GB')

In [ ]:
list(dt.children)

In [ ]:
dt['sweep_0'].ds.DBZH.chunk()

This is quite a large dataset. We can "load" it in memory using lazy evaluation provided by `zarr`, `Xarray`, and `datatree`.

In [ ]:
print(f"Dataset starts at: {dt['sweep_0'].times.min().values} and finishes at {dt['sweep_0'].times.max().values}")

## 1.1 Ploting radar reflectivity on 2023-04-07 at 4:25 UTC
We can use `xarray.plot` functionality to create a reflectivity plot

In [ ]:
dt['sweep_0'].sel(times='2023-04-07 4:25', method='nearest').DBZH.plot(x='x', y='y', 
                                                                       vmin=-10, 
                                                                       vmax=60, 
                                                                       cmap='pyart_ChaseSpectral')

## 1.2 QPE using Marshall and Parmer (1948) relationship. 

In [ ]:
%%time
r_total = (0.0376 * dt['sweep_0'].DBZH ** 0.6112 * 5 / 12).sum('times')

We can now visualize the total amount of rainfall rate during the 72 hours


In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
p = ax.pcolormesh(dt['sweep_0'].x.values, dt['sweep_0'].y.values, r_total)
fig.colorbar(p, ax=ax, label='Accumulated RR [mm]')
ax.set_title("accumulated rain in 72 hours")

# 2. Accesing Radar data by sweeps

Loading and requesting all radar data might take a while to read from the source (see section 1). However, sometimes we only look for a specific sweep instead of "loading" all radar data into memory. `xarray datatree` scheme allows us to load by sweeps by specifying the path to the desired sweep e.g., `radar.zarr/sweep_0`. This will now substantially reduce the "loading" time and the size of the dataset.


In [ ]:
%%time
ds = xr.open_zarr(f"{path}/sweep_1")

In [ ]:
ds

In [ ]:
print(f'size: {ds.nbytes / (1024 ** 3)} GB')

## 2.1 Ploting radar reflectivity

Similarly to section 1.1, we can plot the radar reflectivity. 

In [ ]:
ds.sel(times='2023-04-07 4:25', method='nearest').DBZH.plot(x='x', y='y', 
                                                             vmin=-10, 
                                                             vmax=60, 
                                                             cmap='pyart_ChaseSpectral')

## 2.2 QPE 

We can perform exactly the same rainfall accumulation calculation and this will will be way much faster than using the whole dataset.

In [ ]:
%%time
r_total_sweep = (0.0376 * ds.DBZH ** 0.6112 * 5).sum('times')

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
p = ax.pcolormesh(ds.x.values, ds.y.values, r_total_sweep)
fig.colorbar(p, ax=ax, label='Accumulated RR [mm]')
ax.set_title("accumulated rain in 72 hours")